Python pgm to read jason file

In [232]:
import json

In [43]:
f = open('Input.json')

In [44]:
jdata = json.load(f)

In [45]:
print(jdata)

{'emp_details': [{'emp_name': 'chetan', 'email': 'chetan@email.com', 'profile': 'Dev'}, {'emp_name': 'harshita', 'email': 'harshita@email.com', 'profile': 'Data'}]}


In [46]:
for i in jdata['emp_details']:
    print(i)

{'emp_name': 'chetan', 'email': 'chetan@email.com', 'profile': 'Dev'}
{'emp_name': 'harshita', 'email': 'harshita@email.com', 'profile': 'Data'}


In [233]:
#f2 = open('trans_1.json')
f2 = open('transactions.json')

In [234]:
jdata2 = json.load(f2)

In [235]:
print(list(jdata2)[0:10])
print(len(jdata2))

['T-17OO0GJUTS', 'T-PE31IN24UI', 'T-87OZC760LE', 'T-EA6D1VQJ9D', 'T-WQNEP49N77', 'T-P6Q9WUWNKF', 'T-33M258Y1VB', 'T-DROKGLLWWW', 'T-TT4U7T1L2E', 'T-UONFQNTPTP']
27545


In [37]:
for i in jdata2['T-17OO0GJUTS']:
    print(i)
print('Printing subitems:-------------')
for j in jdata2['T-17OO0GJUTS']['line_items']:
        print(j)

customer_email
order_date
line_items
Printing subitems:-------------
DARKMAGENTA-2Q
BLANCHEDALMOND-1Q


In [78]:
for i in jdata2:
    print(i)
    print(jdata2[i]['customer_email'])
    for j in jdata2[i]['line_items']:
        print(j)
        print(jdata2[i]['line_items'][j]['quantity'])


T-17OO0GJUTS
dorothygruber@gmail.com
DARKMAGENTA-2Q
5
BLANCHEDALMOND-1Q
1
T-PE31IN24UI
raymondsullivan@gmail.com
RED-5G
3
SIENNA-5G
3


In [236]:
#Create connection with MYSQL
import mysql.connector
from mysql.connector import errorcode

In [237]:
try:
    cnx = mysql.connector.connect(user='root', password='Chetan',
                                host='localhost', database='edw')
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
            print('Invalid id or password')
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
            print(err.errno)
            print('Wrong dbname or db does not exists')
    else:
            print(err)
else:
        print('Connection to db successful')
        cursor = cnx.cursor()

Connection to db successful


In [238]:
add_tran = ("INSERT INTO `edw`.`transaction_stg` "
               "(`tranid`, `customer_email`, `order_date`, `line_num`, `product`, `quantity`, `color`, `item_size`, `total_price`)"
               "VALUES (%(tranid)s, %(customer_email)s, %(order_date)s, %(line_num)s, %(product)s, %(quantity)s, %(color)s, %(item_size)s, %(total_price)s)"
            )

In [239]:
cn = 0
for i in jdata2:
    cn = 1
    for j in jdata2[i]['line_items']:
        try:
            data_tran = {"tranid": i, "customer_email": jdata2[i]['customer_email'], "order_date": jdata2[i]['order_date'],
                         "line_num": cn, "product": j, "quantity": jdata2[i]['line_items'][j]['quantity'], 
                         "color": jdata2[i]['line_items'][j]['color'], "item_size": jdata2[i]['line_items'][j]['item_size'], 
                         "total_price": jdata2[i]['line_items'][j]['total_price']
                        }
            cursor.execute(add_tran, data_tran)
        except mysql.connector.Error as err:
            print(err)
        else: 
            cn = cn + 1

In [240]:
cnx.commit()
cursor.close()
cnx.close()

**ALTERNATE APPROACH WITH NORMALIZED TABLES**

In [187]:
add_tran2 = ("INSERT INTO `edw`.`transaction_2` "
               "(`tranid`, `customer_email`, `order_date`)"
               "VALUES (%(tranid)s, %(customer_email)s, %(order_date)s)"
            )

In [188]:
add_prod = ("INSERT INTO `edw`.`product` "
               "(`tranid`, `product`, `quantity`, `color`, `item_size`, `total_price`)"
               "VALUES (%(tranid)s, %(product)s, %(quantity)s, %(color)s, %(item_size)s, %(total_price)s)"
            )

In [189]:
for i in jdata2:
    data_tran2 = {"tranid": i, "customer_email": jdata2[i]['customer_email'], "order_date": jdata2[i]['order_date']}
    cursor.execute(add_tran2, data_tran2)
        
    for j in jdata2[i]['line_items']:
        try:
            data_prod = {"tranid": i, "product": j, "quantity": jdata2[i]['line_items'][j]['quantity'], 
                         "color": jdata2[i]['line_items'][j]['color'], "item_size": jdata2[i]['line_items'][j]['item_size'], 
                         "total_price": jdata2[i]['line_items'][j]['total_price']
                        }
            cursor.execute(add_prod, data_prod)
            
        except mysql.connector.Error as err:
                print(err)  
                cnx.commit()
                cursor.close()
                cnx.close()

In [210]:
cnx.commit()
cursor.close()
cnx.close()

MySQLInterfaceError: MySQL server has gone away

In [201]:
insr_order = ("insert into edw.order_f(tranid, custid, order_date, line_num, productid, quantity, total_price) "
                "select t.tranid, c.custid, t.order_date, t.line_num, p.productid, t.quantity, t.total_price "
                "from edw.transaction t "
                "inner join edw.customer_dim c on t.customer_email = c.customerEmail "
                "inner join edw.product_dm p   on t.product = p.product "
             )

In [202]:
cursor.execute(insr_order)

In [207]:
trunc_tbl = ("truncate table edw.transaction")

In [208]:
cursor.execute(trunc_tbl)

ProgrammingError: 1146 (42S02): Table 'edw.transaction' doesn't exist

In [241]:
a = 10

In [243]:
print('total rows: ', a)

total rows:  10
